# Benchmark Paper Experiment
```BibTeX
@article{theate2021application,
  title={An application of deep reinforcement learning to algorithmic trading},
  author={Th{\'e}ate, Thibaut and Ernst, Damien},
  journal={Expert Systems with Applications},
  volume={173},
  pages={114632},
  year={2021},
  publisher={Elsevier}
}
```

## Setup Notebook

In [ ]:
import os
import sys
import warnings
import time
import yaml
import pickle
from textwrap import dedent
from enum import Enum
from typing import List
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

from scipy import stats

module_path = os.path.abspath(os.path.join(os.getcwd(), 'utils'))
if module_path not in sys.path:
    sys.path.append(module_path)

warnings.filterwarnings("ignore")

INSTALL_DEPS = True
if INSTALL_DEPS:
    %pip install yfinance==0.2.43
    %pip install openai==1.51.2
IN_KAGGLE = IN_COLAB = False
try:
    import google.colab
    from google.colab import drive

    drive.mount("/content/drive")
    DATA_PATH = "/drive/input/drl-dataset-quant"
    IN_COLAB = True
    print("Colab!")
except:
    IN_COLAB = False
if "KAGGLE_KERNEL_RUN_TYPE" in os.environ and not IN_COLAB:
    print("Running in Kaggle...")
    for dirname, _, filenames in os.walk("/kaggle/input"):
        for filename in filenames:
            print(os.path.join(dirname, filename))
    DATA_PATH = "/kaggle/input/drl-dataset-quant"
    sys.path.insert(1, "/kaggle/usr/lib/drlutil")
    IN_KAGGLE = True
    print("Kaggle!")
elif not IN_COLAB:
    IN_KAGGLE = False
    DATA_PATH = "./data/"
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)
    print("running localhost!")

from tqdm.notebook import tqdm
import logging
from pprint import pprint

from dotenv import load_dotenv
load_dotenv()

%load_ext dotenv

RL_OUTPUT_PATH = os.getenv("RL_OUTPUT_PATH", '/rl_data')
LOGS_PATH = os.getenv("LOGS_PATH", '/logs')
paths = [LOGS_PATH, RL_OUTPUT_PATH]

for path in paths:
    if path and not os.path.exists(path):
        os.makedirs(path)

from paper_utils import TradingSimulator, PerformanceEstimator

## Experiment Parameters

In [ ]:
STARTDATE = '2012-01-01'
SPLITDATE = '2018-01-01'
ENDDATE = '2020-01-01'
TARGET = 'Tesla'
money=100_000.
stateLength = 30
features = ['Close', 'Low', 'High', 'Volume', 'strat_signal_long', 'strat_signal_short']
observationSpace =  (stateLength - 1) * len(features)
actionSpace = 2
bounds = [1, 30]
step = 1
numberOfEpisodes = 18
percentageCosts = [0, 0.1, 0.2]
transactionCosts = percentageCosts[1]/100
simulator = TradingSimulator()


# Run Experiments

## Test Selected Stock and strategy

In [ ]:
rl_output_dir = f'{RL_OUTPUT_PATH}/benchmark'
os.makedirs(rl_output_dir, exist_ok=True)

train_file = f'{rl_output_dir}/{TARGET}_train_results.pkl'
test_file = f'{rl_output_dir}/{TARGET}_test_results.pkl'
train_env_file = f'{rl_output_dir}/{TARGET}_train_env.pkl'
test_env_file = f'{rl_output_dir}/{TARGET}_test_env.pkl'

if os.path.exists(train_file) and os.path.exists(test_file) and os.path.exists(train_env_file) and os.path.exists(test_env_file):
    with open(train_file, 'rb') as f:
        train_results = pickle.load(f)
    with open(test_file, 'rb') as f:
        test_results = pickle.load(f)
    with open(train_env_file, 'rb') as f:
        train_env = pickle.load(f)
    with open(test_env_file, 'rb') as f:
        test_env = pickle.load(f)
else:
    start_train_time = time.time()
    strat, train_env, test_env = simulator.simulateNewStrategy("TDQN",
                                                                TARGET,
                                                                startingDate=STARTDATE,
                                                                endingDate=ENDDATE,
                                                                splitingDate=SPLITDATE,
                                                                verbose=True,
                                                                plotTraining=True,
                                                                rendering=True,
                                                                showPerformance=True,
                                                                saveStrategy=True,
                                                                money=money,
                                                                observationSpace=observationSpace,
                                                                actionSpace=actionSpace,
                                                                stateLength=stateLength,
                                                                bounds=bounds,
                                                                step=step,
                                                                numberOfEpisodes=numberOfEpisodes,
                                                                transactionCosts=transactionCosts)
    analyser = PerformanceEstimator(train_env.data)
    train_results = analyser.getComputedPerformance()
    analyser = PerformanceEstimator(test_env.data)
    test_results = analyser.getComputedPerformance()
    with open(train_file, 'wb') as f:
        pickle.dump(train_results, f)
    with open(test_file, 'wb') as f:
        pickle.dump(test_results, f)
    with open(train_env_file, 'wb') as f:
        pickle.dump(train_env, f)
    with open(test_env_file, 'wb') as f:
        pickle.dump(test_env, f)

pprint(test_results)

In [ ]:
sharpe_df = pd.read_csv(f"./papers/results.csv")
sharpe_df.set_index('Stock', inplace=True)

final_summary_df = pd.DataFrame()

for stock_name, stock in tqdm(STOCKS.items(), disable=True):
    with open(f'./{stock}_bm_train_results.pkl', 'rb') as f:
        test_results = pickle.load(f)
    with open(f'./{stock}_bm_time_results.pkl', 'rb') as f:
        time_results = pickle.load(f)

    metrics_data = {}
    for df in test_results:
        for _, row in df.iterrows():
            metric = row['Metric']
            value = row['Value']
            if metric not in metrics_data:
                metrics_data[metric] = []
            metrics_data[metric].append(value)

    stock_summary = {}

    for metric, values in metrics_data.items():
        data = pd.Series(values)
        mean_val = round(data.mean(), 2)
        std_val = round(data.std(), 2)
        t_stat, p_value = stats.ttest_1samp(data, 0)
        p_value = round(p_value, 2)

        stock_summary[metric] = [mean_val, std_val, p_value]

    stock_df = pd.DataFrame(stock_summary, index=["Mean", "+/-", "P-Value"]).T
    stock_df = stock_df.T.unstack().to_frame().T
    stock_df.index = [stock_name]

    stock_df['Paper'] = sharpe_df.loc[stock_name, 'TDQN'] if stock_name in sharpe_df.index else None
    stock_df['Avg Train Time (s)'] = time_results['avg_train_time']
    stock_df['Avg Test Time (s)'] = time_results['avg_test_time']

    if final_summary_df.empty:
        final_summary_df = stock_df
    else:
        final_summary_df = pd.concat([final_summary_df, stock_df])

final_summary_df.T

In [ ]:
final_summary_df.to_csv('./bm_results.csv')